In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf

In [2]:
data=pd.read_csv("Input_EIR_Test.csv",index_col="Account_ID")

In [3]:
data

,Dates,Loan_Availed,Cost_Avail,Rate,EMI
Account_ID,,,,,
AA12,12/31/2020,5000000,50000,0.08,-4950000
AA12,1/1/2021,5000000,50000,0.08,41822
AA12,2/28/2021,5000000,50000,0.08,41822
AA12,3/31/2021,5000000,50000,0.08,41822
AA12,4/30/2021,5000000,50000,0.08,41822
...,...,...,...,...,...
AA12,8/31/2040,5000000,50000,0.08,41822
AA12,9/30/2040,5000000,50000,0.08,41822
AA12,10/31/2040,5000000,50000,0.08,41822


In [4]:
data['Effect_Amt']=data.Loan_Availed-data.Cost_Avail
print(data['Effect_Amt'])

Account_ID
AA12    4950000
AA12    4950000
AA12    4950000
AA12    4950000
AA12    4950000
         ...   
AA12    4950000
AA12    4950000
AA12    4950000
AA12    4950000
AA12    4950000
Name: Effect_Amt, Length: 241, dtype: int64


In [5]:
cashflow1=[data.Effect_Amt[1]*-1]
cashflow2=data.EMI.to_numpy()
cashflows=np.concatenate((cashflow1,cashflow2))
irr=round(npf.irr(cashflows),4)

print(cashflows)
print(irr)

[-4950000 -4950000    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41822    41822    41822
    41822    41822    41822    41822    41822    41

In [6]:
npf.irr(cashflows)

0.00011407783832551033

In [7]:
import datetime
from scipy import optimize

def xnpv(rate,cashflows):
    chron_order = sorted(cashflows, key = lambda x: x[0])
    t0 = chron_order[0][0]
    return sum([cf/(1+rate)**((t-t0).days/365.0) for (t,cf) in chron_order])

def xirr(cashflows,guess=0.1):
    return optimize.newton(lambda r: xnpv(r,cashflows),guess)

In [8]:
import scipy.optimize

def xnpv(rate, values, dates):
    '''Equivalent of Excel's XNPV function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xnpv(0.1, values, dates)
    -966.4345...
    '''
    if rate <= -1.0:
        return float('inf')
    d0 = dates[0]    # or min(dates)
    return sum([ vi / (1.0 + rate)**((di - d0).days / 365.0) for vi, di in zip(values, dates)])

def xirr(values, dates):
    '''Equivalent of Excel's XIRR function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xirr(values, dates)
    0.0100612...
    '''
    try:
        return scipy.optimize.newton(lambda r: xnpv(r, values, dates), 0.0)
    except RuntimeError:    # Failed to converge?
        return scipy.optimize.brentq(lambda r: xnpv(r, values, dates), -1.0, 1e10)

In [9]:
from pyxirr import xirr
from datetime import date
dates=pd.to_datetime(data.Dates)
xirr(dates,data.EMI)

#print("12/31/2020",-4950000, xirr(date(2010, 12, 29),-4950000))

0.08444035344495061

In [10]:
dates=pd.to_datetime(data.Dates)

In [33]:
pd.date_range('2014-10-25','2019-11-25', 
              freq='MD').strftime("%Y-%m-%d").tolist()

ValueError: Invalid frequency: MD

In [30]:
import datetime
date = datetime.datetime.strptime('2014-11-25', "%Y-%m-%d") + pd.offsets.DateOffset(months=60)
print(date)

2019-11-25 00:00:00


datetime.datetime(2014, 11, 1, 0, 0)

In [39]:
datetime.datetime.strptime('12/30/2020',"%m/%d/%Y").strftime("%Y-%m-%d")

'2020-12-30'